In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader,random_split
import torch.nn.functional as F

In [ ]:
# Define the VGG11 model
class VGG11(nn.Module):
    def __init__(self, num_classes=10):  # Defaulting to 10 classes for MNIST
        super(VGG11, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=3, padding=1),  # Input: 1 channel (grayscale)
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),       # Output: 64 channels

            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),       # Output: 128 channels

            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),       # Output: 256 channels

            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),       # Output: 512 channels

            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),       # Output: 512 channels
        )
        self.classifier = nn.Sequential(
            nn.Linear(512 * 1 * 1, 4096),  # Adjust input size if needed for the dataset
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        # Flatten the tensor
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x


In [ ]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# Data loading and preprocessing
transform = transforms.Compose([
    transforms.Resize((32, 32)),  # Resize to fit AlexNet input size
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
train_set, val_set = random_split(train_dataset, [0.9, 0.1])
train_loader = DataLoader(dataset=train_set, batch_size=2048, shuffle=True)
val_loader = DataLoader(dataset=val_set, batch_size=2048, shuffle=True)

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9.91M/9.91M [00:02<00:00, 3.64MB/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28.9k/28.9k [00:00<00:00, 68.6kB/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1.65M/1.65M [00:07<00:00, 220kB/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4.54k/4.54k [00:00<00:00, 8.69MB/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [ ]:
#Instantiate the VGG11 model
model = VGG11(num_classes=10).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for idx,(images, labels) in enumerate(train_loader):#train
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        if(idx % 5 == 0):
          print(f'batch:{idx} , batch loss:{loss.item()}\n')
        total_loss += loss.item()

    acc=0
    for idx,(images, labels) in enumerate(val_loader): #validation
        images, labels = images.to(device), labels.to(device)
        outputs = torch.argmax(model(images),dim=1)
        comp=torch.eq(labels,outputs).float().to('cpu')
        acc+=torch.sum(comp)/len(comp)

    print(f'Epoch [{epoch + 1}/{num_epochs}], Total loss: {total_loss / len(train_loader):.4f} , Accuracy : {acc/(idx+1)} %')

print("Training complete!")

batch:0 , batch loss:1.2753186225891113

batch:5 , batch loss:1.2636518478393555

batch:10 , batch loss:1.1441742181777954

batch:15 , batch loss:1.0533167123794556

batch:20 , batch loss:1.000481128692627

batch:25 , batch loss:0.844616174697876

Epoch [1/10], Total loss: 1.0791 , Accuracy : 0.6438227295875549 %
batch:0 , batch loss:0.8499715924263

batch:5 , batch loss:0.8145959973335266

batch:10 , batch loss:0.7073031067848206

batch:15 , batch loss:0.7828456163406372

batch:20 , batch loss:0.6111133694648743

batch:25 , batch loss:0.6415795683860779

Epoch [2/10], Total loss: 0.7277 , Accuracy : 0.7794063091278076 %
batch:0 , batch loss:0.5823383927345276

batch:5 , batch loss:0.6215003728866577

batch:10 , batch loss:0.5152417421340942

batch:15 , batch loss:0.4505057632923126

batch:20 , batch loss:0.44711658358573914

batch:25 , batch loss:0.38825684785842896

Epoch [3/10], Total loss: 0.4951 , Accuracy : 0.8931963443756104 %
batch:0 , batch loss:0.38049063086509705

batch:5 , 